In [3]:
import os, sys, boto, mimetypes, zipfile, gzip
from io import StringIO, BytesIO
from optparse import OptionParser

import Quandl #NEED TO MAKE SURE YOU HAVE THE QUANDL.PY CODE IN THE SAME FOLDER
import requests
import json
from collections import Counter

##Using the metadata to extract all of the data file names, we can extract all of the datasets

In [4]:
def makelist(num_pages):
    list_of_pages = []
    for i in range(1,num_pages+1):
        list_of_pages.append(i)
    return list_of_pages

##Make a list depending on the dataset size information. In the links for the metadata, it'll say how many times the dataset needs to be updated. For example, for the trading economics data, the metadata page: https://www.quandl.com/data/SGE/metadata shows how many requests need to be made in the bottom of the page (25 in this case)

In [5]:
#Getting all of the data file names from the metadata files (there are a certain number of pages)
#Each page that we can request has a limit of 300 dataset information, and so there are 25 pages in 
#trading economics database

In [6]:
def get_datasheets(num_pages, url_str):
    list_of_pages_all = makelist(num_pages)
    list_name = []

    for i in list_of_pages_all:
        url = url_str+str(i)+'&auth_token=z6JR1NGSDfmAmGix8Muu'
        response = requests.get(url)
        data = json.loads(response.content)
        for i in range(0,len(data[u'docs'])):
            list_name.append(str(data[u'docs'][i][u'code']))
    return list_name

#Trading Economics: https://www.quandl.com/data/SGE/metadata

In [7]:
list_of_all_trading_economics = get_datasheets(25, 'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=SGE&per_page=300&page=')

In [8]:
list_of_all_trading_economics

['AUTPPIC',
 'THACA',
 'MLTGCP',
 'ALBPOP',
 'CHNGPC',
 'AUTGSG',
 'AUTWAGE',
 'TUNDINV',
 'COGCPIC',
 'TURIP',
 'GBRCARS',
 'TWNEXVOL',
 'BOLIMVOL',
 'GRCGD',
 'IRLDINV',
 'ISRCF',
 'CZEM1',
 'HKGM3',
 'BLZBOT',
 'BLZEXVOL',
 'BGRPPI',
 'HRVIPMOM',
 'EURM1',
 'CZEGD',
 'CZEM0',
 'DNKGFCF',
 'LKACPIC',
 'FRAPPIC',
 'SVNGCP',
 'HUNPPI',
 'ITAGFCF',
 'FRACPICM',
 'ITACPI',
 'ITACINF',
 'JPNCPI',
 'PHLM1',
 'PRTCSP',
 'SVNCPI',
 'SVNCINF',
 'ZAFM3',
 'ZAFEXVOL',
 'ESPCPI',
 'LKACPI',
 'THAPSP',
 'TUREXVOL',
 'TURIMVOL',
 'COMPOP',
 'BGDGAGR',
 'KORCTR',
 'KORSSRC',
 'ARGHOUS',
 'BWADINV',
 'AUSTOUR',
 'BHSCAGDP',
 'BRAWAGE',
 'IRNGCP',
 'POLGD',
 'FRABOT',
 'FRACPIC',
 'PRYGPCP',
 'FINGSP',
 'ZAFFER',
 'PRIG',
 'PRTNO',
 'PERGNP',
 'IRLCF',
 'EURGGR',
 'GRCIP',
 'IDNFER',
 'MKDBOT',
 'BRAGSP',
 'ITASPMI',
 'SDNPOP',
 'SVKRSM',
 'TURG',
 'TGOIR',
 'CANIBOR',
 'GTMGBVL',
 'EURDPINC',
 'IRLPPI',
 'POLM1',
 'PRTJVAC',
 'HKGFER',
 'AUTUNRY',
 'BELUNR',
 'CANLEI',
 'HRVUNRY',
 'CYPUNRY',
 'CZEU

#Exchange International Data: https://www.quandl.com/data/CUR

In [21]:
list_of_all_exchange_international_data = get_datasheets(1, 'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=CUR&per_page=300&page=')

In [22]:
list_of_all_exchange_international_data

['ZWL',
 'ZMW',
 'ZMK',
 'ZAR',
 'YER',
 'XAU',
 'XPF',
 'VND',
 'XAG',
 'XOF',
 'XDR',
 'XAF',
 'UZS',
 'UYU',
 'XCD',
 'WST',
 'VUV',
 'TZS',
 'UGX',
 'VEF',
 'UAH',
 'TRY',
 'TTD',
 'THB',
 'TOP',
 'TND',
 'TJS',
 'TWD',
 'SYP',
 'SRD',
 'SZL',
 'TMT',
 'STD',
 'SVC',
 'SEK',
 'SCR',
 'RUB',
 'SOS',
 'SHP',
 'SAR',
 'SGD',
 'SLL',
 'RWF',
 'SDG',
 'PHP',
 'SBD',
 'PKR',
 'PGK',
 'RSD',
 'RON',
 'PLN',
 'QAR',
 'NIO',
 'OMR',
 'PYG',
 'PEN',
 'PAB',
 'NZD',
 'NPR',
 'NOK',
 'NAD',
 'NGN',
 'MVR',
 'MYR',
 'MZN',
 'MWK',
 'MXN',
 'MUR',
 'LRD',
 'MMK',
 'LBP',
 'MNT',
 'MOP',
 'MRO',
 'LYD',
 'MDL',
 'MKD',
 'KZT',
 'KWD',
 'MGA',
 'LSL',
 'MAD',
 'LKR',
 'LAK',
 'KYD',
 'KPW',
 'KGS',
 'KRW',
 'KMF',
 'KHR',
 'JPY',
 'JEP',
 'KES',
 'IRR',
 'JOD',
 'IQD',
 'ILS',
 'JMD',
 'ISK',
 'IMP',
 'HUF',
 'INR',
 'IDR',
 'HNL',
 'GNF',
 'HKD',
 'HRK',
 'GYD',
 'GEL',
 'HTG',
 'GBP',
 'GTQ',
 'GIP',
 'GHS',
 'FKP',
 'FJD',
 'GMD',
 'EUR',
 'GGP',
 'ETB',
 'ERN',
 'EGP',
 'DZD',
 'CRC',
 'DOP',


#Stevens Continuous Futures: https://www.quandl.com/data/SCF

In [25]:
list_of_all_stevens_continuous_futures = get_datasheets(7,'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=SCF&per_page=300&page=')

In [26]:
list_of_all_stevens_continuous_futures

['ICE_SB1_FN',
 'ICE_G1_EN',
 'CME_ED6_EW',
 'CME_FF1_ER',
 'CME_HG2_OF',
 'CME_DJ1_FR',
 'CME_FV1_FR',
 'CME_DJ1_FF',
 'CME_DJ1_FB',
 'ICE_SB1_OB',
 'CME_DJ2_EN',
 'CME_W4_ON',
 'CME_DJ2_FF',
 'ICE_ATW1_FN',
 'CME_HG1_FR',
 'CME_NG1_FF',
 'CME_CL2_OR',
 'CME_ED6_FB',
 'CME_CL2_FW',
 'CME_HO1_FW',
 'CME_CD2_FW',
 'CME_CD2_FF',
 'CME_NG4_OF',
 'ICE_MP1_ON',
 'CME_CL4_EN',
 'CME_DJ1_FN',
 'ICE_ATW2_FF',
 'CME_NG3_FW',
 'CME_JY1_FW',
 'ICE_CT2_FW',
 'ICE_OJ1_FB',
 'CME_S1_FB',
 'ICE_SB1_FR',
 'CME_CL8_EW',
 'ICE_SB2_FF',
 'CME_CL7_EB',
 'CME_NG7_EB',
 'CME_S6_EN',
 'CME_ED7_ER',
 'CME_ED9_ER',
 'ICE_B7_EN',
 'CME_C1_EN',
 'CME_SM1_FN',
 'CME_ES1_FN',
 'ICE_B11_FN',
 'CME_LC1_FN',
 'ICE_KC2_FW',
 'CME_NQ1_FR',
 'CME_NK1_FR',
 'CME_CL6_EF',
 'ICE_ATW2_EF',
 'CME_SP2_EF',
 'ICE_CT2_OF',
 'CME_CL2_OF',
 'CME_NG1_OF',
 'CME_CD1_OF',
 'CME_CL2_EW',
 'CME_S4_EW',
 'ICE_KC1_EW',
 'ICE_RF1_ON',
 'CME_HG1_OF',
 'CME_ED1_FB',
 'CME_LN2_FN',
 'CME_NK1_FN',
 'CME_CL1_EW',
 'CME_W1_EN',
 'CME_HG2_ER',


#CME: Chicago Mercantile Exchange Futures Data: https://www.quandl.com/data/CME

In [27]:
list_of_all_cme = get_datasheets(294,'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=CME&per_page=300&page=')

In [28]:
list_of_all_cme

['71Q2014',
 'OH2014',
 'HGG2004',
 'LOU2015P83',
 'BRH2014',
 'F5H2014',
 'HXN2014P3_22',
 'AOQ2014P65',
 'PDMU2016',
 'CDG2014D107',
 'GLG2015',
 'SSIZ2014',
 'KCU1997',
 'LOM2014P76',
 'FVH2003',
 'OBU2014C2_81',
 'AON2014C83',
 'PAZ1983',
 'BOK1974',
 'AOU2015C93',
 'LNQ2014C4_55',
 'EHQ2013',
 'OSM2014P104',
 'HGF2001',
 'LOM2014P97',
 'BOU1983',
 'Q6Q2014C575',
 'ON2010',
 'OBK2014P2_65',
 'GIM2004',
 'SOU2014P26_25',
 'BON1994',
 'OGN2014P950',
 'HRF2015',
 'NPF2015',
 'BOK2011',
 'SF2015',
 'LNJ2016P3_6',
 'UOX2017',
 'UOK2017',
 'OHM2014C2_96',
 'Q5Z2014',
 'O1F2015',
 'OHF2015C2_48',
 'FTMZ2018',
 'FTMJ2017',
 'FTMK2017',
 'FTMH2017',
 'FTMN2017',
 'FTMV2017',
 'FTMG2018',
 'RAN2014C2_2',
 'FTMU2018',
 'FTMM2018',
 'FTMV2018',
 'Q5F2015',
 'KWN2008',
 'LBK2001',
 'LBX1999',
 'AOM2014P75',
 'LNV2010',
 'LNV2016P6_4',
 'OGM2017C1765',
 '0FZ2014',
 'PAU1985',
 'FDMJ2016',
 'FDMU2016',
 'FDMQ2018',
 'FDMN2015',
 'FDMU2015',
 'FDMV2015',
 'FDMZ2015',
 'FDMK2015',
 'FDMX2015',
 'FD

#Intercontinental Exchange Futures Data: https://www.quandl.com/data/ICE

In [29]:
list_of_all_ice = get_datasheets(24,'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=ICE&per_page=300&page=')

In [30]:
list_of_all_ice

['PK2005',
 'RSH1987',
 'BZ2006',
 'SBK1967',
 'ATWQ2013',
 'CERX2017',
 'OJH2009',
 'OJK1993',
 'KCU2018',
 'GG2013',
 'KCH1981',
 'CZ2008',
 'OK2019',
 'MPU2014',
 'RSN2003',
 'RSK2010',
 'RSF2012',
 'ATWV2011',
 'AFRX2010',
 'TH2011',
 'ARZ2017',
 'AFRK2013',
 'TQ2012',
 'BPBZ2012',
 'KCU1989',
 'KCU1980',
 'SBH1987',
 'GQ2009',
 'RFM2010',
 'GN1996',
 'MPU2011',
 'CJ2020',
 'ERUG2013',
 'GJ2006',
 'ICNV2015',
 'ZRH2007',
 'RFU2009',
 'NCFF2010',
 'CEUV2015',
 'BG2014',
 'RFH2008',
 'BPBH2009',
 'SBH1981',
 'DXU2012',
 'BZ2013',
 'TFMX2014',
 'YV2012',
 'CCH1986',
 'TFM2007',
 'CJ2014',
 'BF2011',
 'AFRJ2010',
 'MH2013',
 'MG2013',
 'TFMZ2013',
 'TK2013',
 'OZ2009',
 'BN1992',
 'BM2011',
 'DXH2010',
 'BM2010',
 'KCK1986',
 'BN2012',
 'BV2013',
 'ULSH2017',
 'KRUH2014',
 'TFM2014',
 'KCG2017',
 'TFMM2014',
 'AFRZ2007',
 'SBH1964',
 'CCU1978',
 'WAK2013',
 'ICNU2016',
 'SBF2004',
 'AFRJ2011',
 'SBV1980',
 'SYU2005',
 'CCN1985',
 'MG2011',
 'SBU1978',
 'YM2005',
 'OJK1976',
 'OJF1972',

#LIFFE Futures Data: https://www.quandl.com/data/LIFFE

In [31]:
list_of_all_liffe = get_datasheets(8,'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=LIFFE&per_page=300&page=')

In [32]:
list_of_all_liffe

['FTIN2015',
 'FEOH2016',
 'EPRH2016',
 'FEOU2015',
 'PSIH2016',
 'EPRU2015',
 'PSIZ2015',
 'EOBH2016',
 'EMAX2015',
 'PSIM2016',
 'EMAF2016',
 'EOBX2015',
 'EOBF2016',
 'EOBK2016',
 'EOBF2017',
 'EMAQ2015',
 'EOBX2016',
 'EMAF2017',
 'EMAH2017',
 'EMAM2016',
 'EMAH2016',
 'EOBX2017',
 'EMAQ2016',
 'EMAX2016',
 'EMAM2017',
 'EOBF2018',
 'FMXU2015',
 'FEFH2016',
 'FEFU2015',
 'FMXN2015',
 'FMXQ2015',
 'PSIU2015',
 'FEFZ2015',
 'EOBH2018',
 'EOBH2017',
 'EOBK2017',
 'LX2014',
 'ZZ1987',
 'WK2010',
 'SM2001',
 'WH2006',
 'WH2010',
 'WH2008',
 'WH2009',
 'WZ1993',
 'WK2005',
 'WZ2003',
 'WQ2007',
 'WZ2001',
 'WZ2002',
 'WV2007',
 'WZ2011',
 'LK2007',
 'LK2005',
 'ZU1992',
 'LU1995',
 'IQ2006',
 'RCH2005',
 'BXFU2014',
 'ZZ1996',
 'BXFK2014',
 'LU1990',
 'SH2012',
 'FCEH2011',
 'JGBM2007',
 'ZM1992',
 'BXFX2014',
 'CH2015',
 'CK2001',
 'SH2005',
 'IF2003',
 'EPEZ2013',
 'SU1997',
 'IN2001',
 'IZ2000',
 'EONG2016',
 'RZ2015',
 'MFCN2014',
 'FCEH2000',
 'JGBZ2002',
 'WK2001',
 'IK2002',
 'IQ2

#EUREX Futures Data: https://www.quandl.com/data/EUREX

In [33]:
list_of_all_eurex = get_datasheets(6,'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=EUREX&per_page=300&page=')

In [34]:
list_of_all_eurex

['FCGRZ2013',
 'FMMAU2013',
 'FESXH2015',
 'FGBMM2007',
 'FCXEU2014',
 'FEU3F2000',
 'FGBSU2013',
 'FCGRH2014',
 'FCCOZ2019',
 'FMPHZ2013',
 'FEXFH2015',
 'FEU3U2006',
 'FBUTZ2014',
 'FMMAZ2014',
 'FMCOZ2013',
 'FEO1K2014',
 'FEO1V2013',
 'FGBMU2002',
 'FBTPH2015',
 'F2MXH2015',
 'FEU3J2000',
 'FSMPJ2015',
 'FGBMZ2010',
 'FEO1Z2015',
 'FSMPU2015',
 'FPIGJ2016',
 'FHOGM2016',
 'FSMPN2016',
 'FMEYU2014',
 'CONFM2014',
 'FEU3Z2005',
 'FSLIZ2014',
 'FWHYN2015',
 'FWHYF2016',
 'FMEEU2013',
 'FCINN2015',
 'FMMXU2013',
 'FTDXM2015',
 'FESXU2015',
 'FHOGU2015',
 'FHOGF2016',
 'FEU3U2009',
 'FGBMU2008',
 'FMJPU2013',
 'FGBLM2001',
 'FGBLH1995',
 'FDAXU2012',
 'FGBLU2010',
 'FGBLH2012',
 'FERUZ2016',
 'FGBSH2009',
 'FVSV2014',
 'FGBSH2013',
 'FMRSU2014',
 'FBUTV2014',
 'FESXM2015',
 'FCPEQ2015',
 'FMCLH2016',
 'FMJPZ2016',
 'FCAGQ2015',
 'FCLIQ2015',
 'FBUTX2014',
 'FEU3Z2016',
 'FCLIZ2017',
 'FCCOH2014',
 'FDAXZ2002',
 'F2EAZ2014',
 'FMEAU2014',
 'FEXDZ2013',
 'FESXH1999',
 'FFOXM2014',
 'F2MXU

#WIKI Continuous Futures Data: https://www.quandl.com/data/CHRIS

In [76]:
import datetime
def get_datasheets_before_time(num_pages, url_str, year):
    list_of_pages_all = makelist(num_pages)
    list_name = []

    for i in list_of_pages_all:
        url = url_str+str(i)+'&auth_token=z6JR1NGSDfmAmGix8Muu'
        response = requests.get(url)
        data = json.loads(response.content)
        for i in range(0,len(data[u'docs'])):
            slist = (str(data[u'docs'][i][u'from_date'])).split("-")
            if slist[0] != 'None' and int(slist[0]) < year:
                list_name.append(str(data[u'docs'][i][u'code']))
    return list_name

In [77]:
list_of_all_chris = get_datasheets_before_time(13, 'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=CHRIS&per_page=300&page=', 2000)

In [82]:
list_of_all_chris

['LIFFE_C4',
 'CME_CL35',
 'OSE_DJIA1',
 'SHFE_CU5',
 'SHFE_CU7',
 'CME_S6',
 'CME_W1',
 'CME_SP3',
 'CME_SI4',
 'ICE_OJ7',
 'ICE_DX2',
 'TFX_JBA5',
 'ICE_B20',
 'CME_BR2',
 'CME_S7',
 'CME_NQ1',
 'CME_EC2',
 'ICE_RS3',
 'CME_O2',
 'CME_SM5',
 'CME_CL3',
 'CME_SM4',
 'CME_ED14',
 'CME_W2',
 'CME_EY1',
 'CME_LN7',
 'CME_JY2',
 'CME_NK1',
 'CME_PL1',
 'CME_CL7',
 'CME_ED18',
 'CME_ED13',
 'CME_C1',
 'ICE_B24',
 'EUREX_FGBM1',
 'LIFFE_L5',
 'CME_CL17',
 'LIFFE_FCE3',
 'CME_HG10',
 'ICE_RS4',
 'CME_SF1',
 'CME_CL15',
 'CME_HG6',
 'CME_W4',
 'CME_ED28',
 'CME_NG13',
 'SHFE_AL2',
 'CME_O3',
 'CME_GC4',
 'CME_SI5',
 'MGEX_MW6',
 'ICE_KC2',
 'CME_ED26',
 'CME_HO4',
 'ICE_B6',
 'ICE_CT4',
 'CME_NG29',
 'CME_ED15',
 'CME_NG31',
 'CME_KW5',
 'CME_HO2',
 'ICE_B13',
 'ICE_CT6',
 'CME_ED1',
 'ICE_M4',
 'SHFE_CU3',
 'CME_FF3',
 'CME_O1',
 'ICE_KC5',
 'CME_ED16',
 'CME_LC6',
 'CME_MP1',
 'CME_HO3',
 'CME_B34',
 'CME_S2',
 'ICE_B17',
 'CME_CL33',
 'CME_CL32',
 'CME_HG11',
 'ICE_G9',
 'ICE_CC3',
 'CME_E

In [40]:
#LISTS OF THE DATASETS CODE NAMES:
list_of_all_trading_economics
list_of_all_exchange_international_data
list_of_all_stevens_continuous_futures
list_of_all_cme
list_of_all_ice
list_of_all_liffe
list_of_all_eurex
list_of_all_chris

['CME_DK12',
 'CME_7H2',
 'CME_7Y1',
 'CME_MGC2',
 'CME_8I2',
 'LIFFE_C4',
 'CME_V345',
 'CME_I614',
 'CME_NN9',
 'CME_RB1',
 'CME_N923',
 'CME_CL35',
 'CME_HMO7',
 'CME_NP52',
 'CME_T39',
 'CME_L921',
 'ICE_TIB2',
 'CME_DC2',
 'CME_WJ4',
 'CME_WS9',
 'CME_DA6',
 'CME_L19',
 'CME_LN10',
 'CME_TU1',
 'CME_NN27',
 'CME_4P12',
 'CME_N14',
 'CME_I66',
 'CME_FTM5',
 'CME_WCC4',
 'CME_E52',
 'CME_XAK1',
 'CME_QG1',
 'CME_WJ5',
 'CME_MFF2',
 'CME_I519',
 'CME_H222',
 'CME_E432',
 'CME_B614',
 'CME_E443',
 'CME_H230',
 'CME_I617',
 'EUREX_F2CR2',
 'EUREX_FSMP2',
 'LIFFE_FTI1',
 'ASX_VW3',
 'LIFFE_RC5',
 'ASX_US4',
 'LIFFE_EON1',
 'OSE_DJIA1',
 'LIFFE_RC1',
 'SHFE_CU5',
 'EUREX_FMPE2',
 'SHFE_ZN4',
 'OSE_NKVI1',
 'LIFFE_L7',
 'SHFE_AL1',
 'EUREX_FMEY2',
 'SHFE_CU7',
 'SHFE_WR2',
 'MCX_ZNM1',
 'CME_S6',
 'EUREX_FBUT1',
 'CME_N15',
 'CME_BK9',
 'CME_MTF18',
 'CME_W1',
 'CME_BK34',
 'CME_SP3',
 'CME_4M7',
 'CME_BZ14',
 'CME_H450',
 'EUREX_FMMX2',
 'CME_X12',
 'CME_GY1',
 'CME_CS33',
 'CME_SI4',
 '

In [6]:
#EXAMPLE for a certain database:
#for i in list_of_pages_all:
#    url = 'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=SGE&per_page=300&page='+str(i)+'&auth_token=X4GASyhZ_ecozx8hQLXm'
#    response = requests.get(url)
#    data = json.loads(response.content)
#    for i in range(0,len(data[u'docs'])):
#        if str(data[u'docs'][i][u'frequency']) == 'daily': <--------------------- IF STATEMENT EXAMPLE
#            list_of_all_datasheets.append(str(data[u'docs'][i][u'code']))

#If you want to specifically look at (daily, weekly, monthly, annually, quarterly) frequency data, do an if statement to check frequency before appending datasheet name

#Grabbing data and GOOGLE CLOUD STORAGE STUFF:

##******THINGS TO CHANGE: id and access key with the seldn google project's id and key, as well as the bucket name/location******

In [9]:
gs_access_key_id = 'GOOGZJ7NUP7VFYAGYTRL'
gs_secret_access_key = 'srwX2b70+Wa1afRe9kR22qH4XulBkrwCYZMNklmI'

In [10]:
conn = boto.connect_gs(gs_access_key_id,gs_secret_access_key)

In [11]:
#Make a bucket beforehand, and use the name as bucket_name

In [12]:
buckets = conn.get_all_buckets()
list_of_buckets = []
for bucket in buckets:
  list_of_buckets.append(bucket.name)
list_of_buckets[0]

u'pushing_quandl'

In [13]:
bucket_name = list_of_buckets[0]
bucket = conn.get_bucket(bucket_name)

In [14]:
COMPRESSIBLE = [ 'text/plain', 'text/csv', 'application/xml',
                'application/javascript', 'text/css' ]
os.environ['AWS_ACCESS_KEY_ID'] = 'Your AWS access key id goes here'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'Your AWS secret access key goes here'

In [24]:
#Creates the datasheet folder on laptop:
newpath = r'DATASHEETS' 
if not os.path.exists(newpath): os.makedirs(newpath)

In [21]:
def push_to_cloud(data_code_name, list_of_all_datasheets):
    data_code = data_code_name

    for i in range(len(list_of_all_datasheets)):
        data = Quandl.get(data_code+'/'+str(list_of_all_datasheets[i]),authtoken="z6JR1NGSDfmAmGix8Muu")
        data.to_csv('DATASHEETS/'+data_code+'_'+str(list_of_all_datasheets[i])+'.csv')
        content = open('DATASHEETS/'+data_code+'_'+str(list_of_all_datasheets[i])+'.csv')
        name = data_code+'_'+str(list_of_all_datasheets[i])+'.csv'
        key = bucket.new_key(name)
        type, encoding = mimetypes.guess_type(name)
        type = type or 'application/octet-stream'
        headers = { 'Content-Type': type, 'x-amz-acl': 'public-read' }
        states = [type]

        if type in COMPRESSIBLE:
            headers['Content-Encoding'] = 'gzip'
            compressed = BytesIO()
            gz = gzip.GzipFile(filename=name, fileobj=compressed, mode='w')
            gz.writelines(content)
            gz.close()
            content.close
            content = BytesIO(compressed.getvalue())
            states.append('gzipped')
        states = ', '.join(states)
        print '- %s =&gt; %s (%s)' % (name, key.name, states)
        key.set_contents_from_file(content, headers)
        content.close();

In [ ]:
#USING THE FUNCTION, ALL OF THE CALLS:
push_to_cloud('SGE',list_of_all_trading_economics)
push_to_cloud('CUR', list_of_all_exchange_international_data)
push_to_cloud('SCF', list_of_all_stevens_continuous_futures)
push_to_cloud('CME', list_of_all_cme)
push_to_cloud('ICE', list_of_all_ice)
push_to_cloud('LIFFE', list_of_all_liffe)
push_to_cloud('EUREX', list_of_all_eurex)
push_to_cloud('CHRIS', list_of_all_chris)

#Getting From-Date Count: Just playing around code (Ignore this part, it's been implemented)

In [25]:
def get_datasheets_dates(num_pages, url_str):
    list_of_pages_all = makelist(num_pages)
    list_name = []

    for i in list_of_pages_all:
        url = url_str+str(i)+'&auth_token=z6JR1NGSDfmAmGix8Muu'
        response = requests.get(url)
        data = json.loads(response.content)
        for i in range(0,len(data[u'docs'])):
            list_name.append(str(data[u'docs'][i][u'from_date']))
    return list_name

In [71]:
list_of_all_chris_dates = get_datasheets_dates(13, 'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=CHRIS&per_page=300&page=')

In [29]:
list_of_all_chris_dates

['2013-11-18',
 '2014-02-18',
 '2014-03-05',
 '2014-03-05',
 '2014-02-18',
 '1993-09-01',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2005-10-03',
 '2014-02-18',
 '1990-12-19',
 '2014-02-18',
 '2014-02-24',
 '2014-02-18',
 '2014-02-18',
 '2011-12-05',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2011-10-03',
 '2014-02-18',
 '2014-12-24',
 '2007-10-01',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-03-05',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '2013-09-09',
 '2013-09-09',
 '2013-12-23',
 '2013-10-09',
 '2013-10-08',
 '2013-10-09',
 '2013-10-08',
 '1943-09-28',
 '2001-05-22',
 '1999-06-10',
 '2013-09-09',
 '2007-06-08',
 '2013-10-16',
 '2015-01-02',
 '2013-10-16',
 '2013-09-09',
 '1999-06-10',
 '2009-06-09',
 '2013-12-04',
 '1959-11-19',
 '2013-09-09',
 '2014-02-18',
 '2014-02-18',
 '2014-02-18',
 '1959-07-

In [79]:
import datetime
new_list_dates = []

for i in range(len(list_of_all_chris_dates)):
    slist = list_of_all_chris_dates[i].split("-")
    if slist[0] != 'None' and int(slist[0]) < 2000:
        new_list_dates.append(int(slist[0]))
print new_list_dates

[1993, 1990, 1943, 1999, 1999, 1959, 1959, 1982, 1964, 1967, 1985, 1992, 1993, 1995, 1960, 1999, 1999, 1996, 1959, 1959, 1983, 1959, 1989, 1959, 1995, 1970, 1976, 1990, 1969, 1983, 1992, 1989, 1959, 1998, 1999, 1990, 1984, 1999, 1959, 1996, 1975, 1984, 1959, 1959, 1993, 1992, 1999, 1959, 1974, 1963, 1994, 1975, 1993, 1979, 1990, 1959, 1995, 1989, 1995, 1984, 1979, 1993, 1960, 1982, 1997, 1999, 1988, 1959, 1973, 1989, 1965, 1995, 1979, 1998, 1959, 1993, 1991, 1991, 1960, 1990, 1970, 1987, 1992, 1988, 1996, 1997, 1963, 1993, 1967, 1986, 1983, 1995, 1975, 1991, 1991, 1983, 1990, 1967, 1967, 1999, 1996, 1969, 1990, 1999, 1960, 1999, 1959, 1974, 1982, 1983, 1959, 1965, 1981, 1969, 1992, 1977, 1959, 1990, 1993, 1993, 1983, 1990, 1984, 1964, 1969, 1990, 1987, 1999, 1979, 1964, 1976, 1994, 1993, 1963, 1990, 1990, 1984, 1998, 1991, 1961, 1995, 1999, 1974, 1967, 1982, 1990, 1995, 1974, 1984, 1968, 1959, 1959, 1970, 1988, 1970, 1982, 1998, 1998, 1983, 1995, 1971, 1998, 1994, 1975, 1959, 1996, 199